<a href="https://colab.research.google.com/github/Rojan116/AI_works/blob/master/characterwise_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch import nn
import numpy as np
import torch.nn.functional as F

In [0]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
  text = f.read()

In [3]:
#test
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [0]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [5]:
encoded[:100]

array([32, 29, 31, 25, 55, 68, 10, 30, 17, 13, 13, 13, 70, 31, 25, 25, 81,
       30, 48, 31, 65, 26,  2, 26, 68, 15, 30, 31, 10, 68, 30, 31,  2,  2,
       30, 31,  2, 26, 66, 68, 79, 30, 68, 44, 68, 10, 81, 30, 58, 21, 29,
       31, 25, 25, 81, 30, 48, 31, 65, 26,  2, 81, 30, 26, 15, 30, 58, 21,
       29, 31, 25, 25, 81, 30, 26, 21, 30, 26, 55, 15, 30, 59, 80, 21, 13,
       80, 31, 81, 73, 13, 13, 33, 44, 68, 10, 81, 55, 29, 26, 21])

In [0]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [0]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

In [8]:
print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [0]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [0]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [11]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[32 29 31 25 55 68 10 30 17 13]
 [15 59 21 30 55 29 31 55 30 31]
 [68 21  1 30 59 10 30 31 30 48]
 [15 30 55 29 68 30 52 29 26 68]
 [30 15 31 80 30 29 68 10 30 55]
 [52 58 15 15 26 59 21 30 31 21]
 [30  5 21 21 31 30 29 31  1 30]
 [54 34  2 59 21 15 66 81 73 30]]

y
 [[29 31 25 55 68 10 30 17 13 13]
 [59 21 30 55 29 31 55 30 31 55]
 [21  1 30 59 10 30 31 30 48 59]
 [30 55 29 68 30 52 29 26 68 48]
 [15 31 80 30 29 68 10 30 55 68]
 [58 15 15 26 59 21 30 31 21  1]
 [ 5 21 21 31 30 29 31  1 30 15]
 [34  2 59 21 15 66 81 73 30 49]]


In [12]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [0]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [15]:

# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [16]:

batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.2385... Val Loss: 3.1875
Epoch: 1/20... Step: 20... Loss: 3.1531... Val Loss: 3.1328
Epoch: 1/20... Step: 30... Loss: 3.1424... Val Loss: 3.1235
Epoch: 1/20... Step: 40... Loss: 3.1095... Val Loss: 3.1195
Epoch: 1/20... Step: 50... Loss: 3.1427... Val Loss: 3.1172
Epoch: 1/20... Step: 60... Loss: 3.1137... Val Loss: 3.1140
Epoch: 1/20... Step: 70... Loss: 3.1028... Val Loss: 3.1094
Epoch: 1/20... Step: 80... Loss: 3.1112... Val Loss: 3.1003
Epoch: 1/20... Step: 90... Loss: 3.0988... Val Loss: 3.0828
Epoch: 1/20... Step: 100... Loss: 3.0445... Val Loss: 3.0250
Epoch: 1/20... Step: 110... Loss: 2.9501... Val Loss: 2.9228
Epoch: 1/20... Step: 120... Loss: 2.8204... Val Loss: 2.8145
Epoch: 1/20... Step: 130... Loss: 2.7184... Val Loss: 2.6709
Epoch: 2/20... Step: 140... Loss: 2.6277... Val Loss: 2.5700
Epoch: 2/20... Step: 150... Loss: 2.5563... Val Loss: 2.5131
Epoch: 2/20... Step: 160... Loss: 2.5059... Val Loss: 2.4649
Epoch: 2/20... Step: 170... Loss:

In [0]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)


In [0]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [0]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)
    

In [20]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna and the
painteds the most play of cast as he constantly began to say in
a feelings and a mill of supposition. And had taken away,
and the peasants, who was
talking of her friend, to her, he was
after the back and at the poon broks wearing happy. He had said,
belaining his head and had been asserted.

"Yes, yes, I'm gliding. How do you did be mere all that I say
indeed, and that you know the position, was that they can't
say was so are there's the father's arrival to the carpering of her? I will
go to almost the same man who has been in
love, was starting out who have a door and the children."

"Oh, you're doubt, but is've seen it. It's not to great abratting."

"Why?" asked Varenka smiling. "You did all see where the
chest was is three things, and I'll say, I can't let me.
And I am saying."

Stepan Arkadyevitch's secretary and the capitated was an
accompare, that in a words observed in spite of his strange, and that he had to
be, when the sound of helpless, and an answer, who came

In [21]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [22]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said, but the same table
of her, who was too thing, and will natural and her heart
already was sorring to the carriage of his way. "You do anything
all one's cleat and plan of its, the delage, but that's that
table," he added as she was as it was so such instant,
and all he had taking his feelings and so to be the more, though she seemed to him to be
asked him, but all that to how it should be dared and askant what the
compary of that the standside and husband sould had never said.

She was not that him was not to step in a stidy of her.

The chief tenderness are, at his simple that the days
were simply for them, and she showed in his brother at her. "How's it arred
with them?" he said: "What are I'm greatly andred as a child," he asked
that he was a carryance of which he heard the sone.

"Well, I cannot to care?" the cropded and stopping about the
ran to Levin, and heard strange in his hand. "Watch you attracting
it, there is a century of a second mind and she would
not see,